# Semantic Similarity Using BERT Transformers

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import transformers
import warnings
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_excel('semantic_similar_Dataset.xlsx')

In [3]:
df.head()

,text,reason,label
0,the app is crashing when i play a vedio,app crashes during playback,1.0
1,but i want to connect it to the tv from one de...,want compatibility with more smart televisions,0.0
2,very helpful when and home working remotley,good app for work,0.0
3,this zoom so called and missed call and mobile...,receiving incorrect phone number message,0.0
4,one of my favorite apps,good for spending time,0.0


In [4]:
df['label'].apply(str) 

0       1.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
8995    1.0
8996    0.0
8997    0.0
8998    0.0
8999    0.0
Name: label, Length: 9000, dtype: object

In [5]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 64
epochs = 1

labels = ["not matches", "matches"]

In [6]:
import pandas as pd
import numpy as np


# Define the proportions for splitting (you can adjust these as needed)
train_ratio = 0.7
valid_ratio = 0.15
test_ratio = 0.15

# Calculate the number of samples for each split
total_samples = len(df)
train_samples = int(train_ratio * total_samples)
valid_samples = int(valid_ratio * total_samples)

# Shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the dataset into training, validation, and testing DataFrames
train_df = df[:train_samples]
valid_df = df[train_samples:train_samples + valid_samples]
test_df = df[train_samples + valid_samples:]



In [7]:
y_train = tf.keras.utils.to_categorical(train_df["label"], num_classes=2)

# Modify validation labels
y_val = tf.keras.utils.to_categorical(valid_df["label"], num_classes=2)

# Modify test labels
y_test = tf.keras.utils.to_categorical(test_df["label"], num_classes=2)

In [8]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [9]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)


In [10]:
# Encoded token ids from BERT tokenizer.
input_ids = tf.keras.layers.Input(
    shape=(max_length,), dtype=tf.int32, name="input_ids"
)
# Attention masks indicates to the model which tokens should be attended to.
attention_masks = tf.keras.layers.Input(
    shape=(max_length,), dtype=tf.int32, name="attention_masks"
)
# Token type ids are binary masks identifying different sequences in the model.
token_type_ids = tf.keras.layers.Input(
    shape=(max_length,), dtype=tf.int32, name="token_type_ids"
)
# Loading pretrained BERT model.
bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
# Freeze the BERT model to reuse the pretrained features without modifying them.
bert_model.trainable = False

bert_output = bert_model.bert(
    input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
)
sequence_output = bert_output.last_hidden_state
pooled_output = bert_output.pooler_output
# Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
bi_lstm = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(64, return_sequences=True)
)(sequence_output)
# Applying hybrid pooling approach to bi_lstm sequence output.
avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
concat = tf.keras.layers.concatenate([avg_pool, max_pool])
dropout = tf.keras.layers.Dropout(0.3)(concat)
output = tf.keras.layers.Dense(2, activation="sigmoid")(dropout)

model = tf.keras.models.Model(
    inputs=[input_ids, attention_masks, token_type_ids], outputs=output
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-5),
    loss="binary_crossentropy",
    metrics=["acc"],
)

model.summary()

# Early Stopping Callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,  # Adjust the patience parameter as needed
    restore_best_weights=True
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_masks[0][0]',    

In [11]:
train_data = BertSemanticDataGenerator(
    train_df[["text", "reason"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    valid_df[["text", "reason"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

In [12]:
 #Unfreeze the bert_model.
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_masks[0][0]',    

In [13]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
    callbacks=[early_stopping]
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


 7/98 [=>............................] - ETA: 1:01:50 - loss: 0.8802 - accuracy: 0.5491

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 9/98 [=>............................] - ETA: 1:00:23 - loss: 0.8414 - accuracy: 0.5521

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

84/98 [========================>.....] - ETA: 9:07 - loss: 0.6744 - accuracy: 0.6445

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


98/98 [==============================] - ETA: 0s - loss: 0.6681 - accuracy: 0.6502  

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will alwa

98/98 [==============================] - 6319s 64s/step - loss: 0.6681 - accuracy: 0.6502 - val_loss: 0.6486 - val_accuracy: 0.6488


In [15]:
# Access training and validation scores
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']  # Use 'accuracy' instead of 'acc'
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']  # Use 'val_accuracy' instead of 'val_acc'

# Display the training and validation scores
print("Training Loss:", train_loss)
print("Training Accuracy:", train_accuracy)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

Training Loss: [0.6680765151977539]
Training Accuracy: [0.6501913070678711]
Validation Loss: [0.6485737562179565]
Validation Accuracy: [0.648809552192688]


In [16]:
test_data = BertSemanticDataGenerator(
    test_df[["text", "reason"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


10/21 [=============>................] - ETA: 2:30 - loss: 0.6155 - accuracy: 0.6984

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


19/21 [==========================>...] - ETA: 27s - loss: 0.6261 - accuracy: 0.6826

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


21/21 [==============================] - 291s 14s/step - loss: 0.6267 - accuracy: 0.6815


[0.6267313361167908, 0.6815476417541504]

In [17]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

10/21 [=============>................] - ETA: 2:40 - loss: 0.6155 - accuracy: 0.6984

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


19/21 [==========================>...] - ETA: 28s - loss: 0.6261 - accuracy: 0.6826

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


21/21 [==============================] - 303s 14s/step - loss: 0.6267 - accuracy: 0.6815
Test Loss: 0.6267313361167908
Test Accuracy: 0.6815476417541504


In [35]:
def check_similarity(text, reason):
    sentence_pairs = np.array([[str(text), str(reason)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    return pred, proba


In [22]:
text = "Two women are observing something together."
reason = "Two ladies are observing something together."
check_similarity(text, reason)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 312ms/step


('not matches', ' 0.66%')

#  Model performance is very poor so we can  use Hyperparameter tuning techniques to improve model performance 

# Fine Tuning with Hyperparameters by using Keras Tuner

In [23]:
pip install keras-tuner


In [24]:
import kerastuner as kt
from kerastuner.tuners import Hyperband

# Define the hypermodel function
def build_hypermodel(hp):
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )

    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")

    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model.bert(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output

    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)

    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])

    # Add dropout layer with hyperparameter tuning
    dropout_rate = hp.Float("dropout_rate", min_value=0.0, max_value=0.5, step=0.1, default=0.3)
    dropout = tf.keras.layers.Dropout(dropout_rate)(concat)

    # Add dense layers with hyperparameter tuning
    dense_units = hp.Int("dense_units", min_value=32, max_value=256, step=32, default=128)
    dense_layer = tf.keras.layers.Dense(dense_units, activation="relu")(dropout)

    output = tf.keras.layers.Dense(2, activation="sigmoid")(dense_layer)

    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    # Compile the model with hyperparameter tuning
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Float("learning_rate", min_value=1e-5, max_value=1e-3, sampling="log")),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    
    return model

# Define the tuner
tuner = Hyperband(
    build_hypermodel,
    objective="val_accuracy",
    max_epochs=10,  # Maximum number of training epochs per trial
    factor=3,  # Reduction factor for the number of models
    directory="tuner_logs",  # Directory for storing tuner logs and checkpoints
    project_name="bert_hyperparameter_tuning",
)

# Define the training data
train_data = BertSemanticDataGenerator(
    train_df[["text", "reason"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)

# Define the validation data
valid_data = BertSemanticDataGenerator(
    valid_df[["text", "reason"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

# Start hyperparameter tuning
tuner.search(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)



Trial 17 Complete [17h 06m 47s]
val_accuracy: 0.8816964030265808

Best val_accuracy So Far: 0.8816964030265808
Total elapsed time: 1d 17h 28m 39s

Search: Running Trial #18

Value             |Best Value So Far |Hyperparameter
0.1               |0.2               |dropout_rate
192               |128               |dense_units
0.00027038        |0.00097636        |learning_rate
10                |10                |tuner/epochs
4                 |4                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
2                 |2                 |tuner/round
0013              |0012              |tuner/trial_id



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 5/10
 1/98 [..............................] - ETA: 1:07:43 - loss: 0.3186 - accuracy: 0.8750

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 2/98 [..............................] - ETA: 30:24 - loss: 0.3321 - accuracy: 0.8750  

...

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


70/98 [====================>.........] - ETA: 8:08 - loss: 0.3524 - accuracy: 0.8444

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


78/98 [======================>.......] - ETA: 5:50 - loss: 0.3514 - accuracy: 0.8435

KeyboardInterrupt: 

# I interrupted Kernal as it was taking too long time to run But, model performance was good with val_acc = 87% 

In [29]:
# Create the final model with the best hyperparameters

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
print(f"Learning Rate: {best_hps.get('learning_rate')}")
print(f"Dropout Rate: {best_hps.get('dropout_rate')}")
print(f"Dense Units: {best_hps.get('dense_units')}")




Best Hyperparameters:
Learning Rate: 0.0009763649814155504
Dropout Rate: 0.2
Dense Units: 128


In [30]:
final_model = build_hypermodel(best_hps)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [32]:
# Train the final model on the entire training dataset
final_model.fit(
    train_data,
    validation_data=valid_data,
    epochs=10,
    use_multiprocessing=True,
    workers=-1,
)


Epoch 1/10
 2/98 [..............................] - ETA: 27:08 - loss: 0.4291 - accuracy: 0.7812

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 7/98 [=>............................] - ETA: 25:49 - loss: 0.4529 - accuracy: 0.7835

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


25/98 [======>.......................] - ETA: 21:05 - loss: 0.4569 - accuracy: 0.7781

...

80/98 [=======================>......] - ETA: 5:22 - loss: 0.1849 - accuracy: 0.9268

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


98/98 [==============================] - ETA: 0s - loss: 0.1906 - accuracy: 0.9246 

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


98/98 [==============================] - 2086s 21s/step - loss: 0.1906 - accuracy: 0.9246 - val_loss: 0.3295 - val_accuracy: 0.8787


In [33]:
# Evaluate the final model on the test dataset
test_data = BertSemanticDataGenerator(
    test_df[["text", "reason"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 740e0f6b-7ba7-404d-bf95-c10c4ca8caf0)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt


In [34]:
test_loss, test_accuracy = final_model.evaluate(test_data, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


10/21 [=============>................] - ETA: 2:44 - loss: 0.2967 - accuracy: 0.8859

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


19/21 [==========================>...] - ETA: 30s - loss: 0.3073 - accuracy: 0.8783

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


21/21 [==============================] - 319s 15s/step - loss: 0.3083 - accuracy: 0.8772
Test Loss: 0.3083288073539734
Test Accuracy: 0.8772321343421936


In [36]:
def check_similarity(text, reason): #For manual testing to see whether model can perdict similarity between sentences
    sentence_pairs = np.array([[str(text), str(reason)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = final_model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    return pred, proba


In [47]:
text = "this app is good ."
reason = " I am using good app."
check_similarity(text, reason)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 398ms/step


('matches', ' 0.74%')